In [1]:
pip install OpenEXR numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import OpenEXR
import Imath
import numpy as np
import os

def get_exr_info(exr_file):
    file = OpenEXR.InputFile(exr_file)
    header = file.header()
    
    # Get image dimensions
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    
    # Get the number of channels
    channels = file.header()['channels'].keys()
    num_channels = len(channels)
    
    return width, height, num_channels, channels

# Folder path to your images
image_folder = '/kaggle/input/kaist-spectral/Dataset'

# Loop through all images and check their dimensions and channels
for image_name in os.listdir(image_folder):
    if image_name.endswith('.exr'):
        image_path = os.path.join(image_folder, image_name)
        try:
            width, height, num_channels, channels = get_exr_info(image_path)
            print(f'{image_name} - Width: {width}, Height: {height}, Channels: {num_channels}')
        except Exception as e:
            print(f"Failed to process {image_name}: {e}")

scene03_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene01_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene17_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene12_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene13_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene11_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene04_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene02_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene10_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene19_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene25_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene22_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene23_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene29_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene28_reflectance.exr - Width: 3376, Height: 2704, Channels: 34
scene26_re

In [5]:
def load_exr_to_pixel_data(exr_file):
    file = OpenEXR.InputFile(exr_file)
    header = file.header()

    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    channels = header['channels'].keys()

    # Define the pixel type (float)
    pt = Imath.PixelType(Imath.PixelType.FLOAT)

    # Read each channel's data
    pixel_data = []
    for channel in channels:
        #convert raw pixel data into floating point numbers
        channel_data = np.frombuffer(file.channel(channel, pt), dtype=np.float32)
        channel_data = channel_data.reshape((height, width))  # Reshape into 2D
        pixel_data.append(channel_data)

    # Stack all the channel data into a (height, width, num_channels) array
    pixel_data = np.stack(pixel_data, axis=-1)

    # Flatten the image into an array of shape (height * width, num_channels)
    pixel_data = pixel_data.reshape(-1, len(channels))  # Each pixel becomes a row

    return pixel_data

image_folder = '/kaggle/input/kaist-spectral/Dataset'

all_pixel_data = []

# Loop through all .exr images and extract pixel data
for image_name in os.listdir(image_folder):
    if image_name.endswith('01_reflectance.exr'):
        image_path = os.path.join(image_folder, image_name)
        try:
            pixel_data = load_exr_to_pixel_data(image_path)
            all_pixel_data.append(pixel_data)
        except Exception as e:
            print(f"Failed to process {image_name}: {e}")

# Concatenate pixel data from all images
all_pixel_data = np.concatenate(all_pixel_data, axis=0)

print(f"Shape of pixel data: {all_pixel_data.shape}")

Shape of pixel data: (9128704, 34)


In [9]:
portion = all_pixel_data[:1000000]

In [10]:
portion.shape

(1000000, 34)

In [11]:
np.save("1mil_pixel_data.npy", portion)